In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage.interpolation import shift

F:\ML\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
MNIST_PATH = os.path.join("datasets", "MNIST")
FILE_NAME = "train.csv"

In [3]:
def load_mnist_data(mnist_path=MNIST_PATH, file_name = FILE_NAME):
    csv_path = os.path.join(mnist_path, file_name)
    return pd.read_csv(csv_path)

In [4]:
def random_batch(X_train, y_train, batch_size):
    rnd_index = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_index, :]
    y_batch = y_train[rnd_index]
    return X_batch, y_batch

In [5]:
mnist = load_mnist_data()

In [6]:
X_train, X_val, y_train, y_val = train_test_split(mnist.values[:, 1:], mnist.values[:, 0], test_size = 0.2, random_state = 42)

In [7]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)

F:\ML\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [73]:
tf.reset_default_graph()

In [74]:
n_steps = 28
n_inputs = 28
n_neurons = 200
n_outputs = 10
n_layers = 3
learning_rate = 0.001
momentum = 0.99

In [75]:
X = tf.placeholder(tf.float32, shape=(None, n_steps, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")
keep_prob = tf.placeholder_with_default(1.0, shape=())

In [76]:
with tf.variable_scope("rnn", initializer = tf.contrib.layers.variance_scaling_initializer()):
    cells = [tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation=tf.nn.elu) for layer in range(n_layers)]
    cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob = keep_prob) for cell in cells]
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells_drop)
    outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

states_concat = tf.concat(axis=1, values=states)
logits = tf.layers.dense(states_concat, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits) 

loss = tf.reduce_mean(xentropy, name="loss")
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = momentum, use_nesterov=True)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [77]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [78]:
X_val = X_val.reshape((-1, n_steps, n_inputs))

In [ ]:
n_epochs = 100
batch_size = 1000
n_batches = int(np.ceil(X_train.shape[0]/ batch_size))
train_keep_prob = 0.5

best_accuracy = 0

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(n_batches):
            X_batch, y_batch = random_batch(X_train, y_train, batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run([training_op, extra_update_ops], feed_dict={keep_prob: train_keep_prob, X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_val, y: y_val})
        print(epoch, "Правильность при обучении:", acc_train,
                     "Правильность при проверке:", acc_test)
        
        if acc_test > best_accuracy:
            best_accuracy = acc_test
            save_path = saver.save(sess, "./mnist/best_model_final4/best_mnist_model_final.ckpt")
            print("Найдена лучшая модель на ", epoch+1, " итерации с точностью при проверке:", acc_test)

0 Правильность при обучении: 0.492 Правильность при проверке: 0.5004762
Найдена лучшая модель на  1  итерации с точностью при проверке: 0.5004762
1 Правильность при обучении: 0.7 Правильность при проверке: 0.7019048
Найдена лучшая модель на  2  итерации с точностью при проверке: 0.7019048
2 Правильность при обучении: 0.788 Правильность при проверке: 0.80833334
Найдена лучшая модель на  3  итерации с точностью при проверке: 0.80833334
3 Правильность при обучении: 0.867 Правильность при проверке: 0.8597619
Найдена лучшая модель на  4  итерации с точностью при проверке: 0.8597619
4 Правильность при обучении: 0.897 Правильность при проверке: 0.8877381
Найдена лучшая модель на  5  итерации с точностью при проверке: 0.8877381
5 Правильность при обучении: 0.91 Правильность при проверке: 0.9033333
Найдена лучшая модель на  6  итерации с точностью при проверке: 0.9033333
6 Правильность при обучении: 0.923 Правильность при проверке: 0.91714287
Найдена лучшая модель на  7  итерации с точностью пр

79 Правильность при обучении: 0.989 Правильность при проверке: 0.98011905
80 Правильность при обучении: 0.988 Правильность при проверке: 0.98
81 Правильность при обучении: 0.989 Правильность при проверке: 0.98333335
Найдена лучшая модель на  82  итерации с точностью при проверке: 0.98333335
82 Правильность при обучении: 0.99 Правильность при проверке: 0.98035717
83 Правильность при обучении: 0.991 Правильность при проверке: 0.9817857
84 Правильность при обучении: 0.99 Правильность при проверке: 0.9784524
85 Правильность при обучении: 0.988 Правильность при проверке: 0.98190475
86 Правильность при обучении: 0.994 Правильность при проверке: 0.98202384
